In [1]:
import sys,os
sys.path.append('../OneClickTraining/')
sys.path.append('../Others/')
from all_model_py import *
from others_py import *
import pickle
import pandas as pd

In [2]:
models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../savedModel/{}/{}.pickle'
model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一遍')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.639 seconds.
Prefix dict has been built succesfully.


In [3]:
import pickle

class ClassifierBase:
    def load_model(self, **model_path):
        pass
            
            
####### classifier 0
####### InitClassifier

class InitClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to initialize conversation'
        self.label_explain = {0: 'next'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        result_dict = {'label':0,'ptp_time':None}
        return result_dict
    

    
    
class StopClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to handle stop logic'
        self.label_explain = {0: 'stop'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        
        result_dict = {'label':0,'ptp_time':None}
        return result_dict

model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()

In [4]:



class Node:
    def __init__(self, node_name, msg_path=None):
        self.name = node_name
        self.entry_counter = 0
        self._load_message(msg_path)
        print('{} is initialized'.format(self.name))

        
        
    def summary(self):
        return {'node_name': self.name, 
                'description':self.describe, 
                'class_name':self.__class__.__name__, 
                'model': self.model_name}
    
    
    
    def process(self, sentence, model_dict):
        model = model_dict[self.model_name]
        clf = model.classify(sentence)
        
        self.output_label = clf['label']
        # jump trigger
        if self.output_label == 1 and self.entry_counter >=2: 
            self.output_label = 1001
        self.ptp_time = clf.get('ptp_time')
        return self.output_label, self.ptp_time
    
    
    def _load_message(self, msg_path):
        self.messages = pd.read_csv(msg_path, encoding='utf8')
        self.messages = self.messages[self.messages['node_name'] == self.name]
        self.messages.label = self.messages.label.astype('int')
        self.messages.sentiment = self.messages.sentiment.astype('int')
        
        
    def get_response(self, label, sentiment=1):
        """
        return response by label
        """
        df = self.messages[(self.messages.label == label) & (self.messages.sentiment == sentiment)]
        response = df.message.values[0]
        self.entry_counter += 1
        return response

        
        

        
###################### Node 0  #########################

    
class S1_N0(Node):
    def __init__(self, msg_path):
        super().__init__('s0', msg_path)
        self.describe = 'Init node'
        self.model_name = 'InitClassifier'

        
###################### Node 1  #########################
class S1_N1(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n1_identity_q', msg_path)
        self.describe = 'Verify Identify'
        self.model_name = 'IDClassifier'
        
        
###################### Node 1  #########################
class S1_N2(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n2_confirmLoan_q', msg_path)
        self.describe = 'Verify Identify'
        self.model_name = 'ConfirmLoan'
                

                
#######################  Node 2  #############################        
class S1_N5(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n5_ifAcquainted_q', msg_path)
        self.describe = 'Ask if know debtor'
        self.model_name = 'IfKnowDebtor'
        
        
##########################  Node 3  ##########################        
class S1_N15(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n15_verifyWill_q', msg_path)
        self.describe = 'Verify willing to pay'
        self.model_name = 'WillingToPay'
                
       

        
#########################  Node 7  ###########################        
class S1_N25(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n25_cutDebt_q', msg_path)
        self.describe = 'ask if accept less amount'
        self.model_name = 'CutDebt'
        
        
        
                
        
#########################  Node 8  ###########################        
class S1_N32(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n32_splitDebt_q', msg_path)
        self.describe = 'ask if accept installment'
        self.model_name = 'Installment'
        
        
        
############################## STOP NODE ########################
class NodeStop(Node):
    def __init__(self, node_name, msg_path):
        super().__init__(node_name, msg_path)
        self.model_name = 'StopClassifier'



class S1_N101(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n101_ifAcquainted_s', msg_path)
        self.describe = 'inform phone recipient'

        
        
class S1_N102(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n102_ifAcquainted_s', msg_path)
        self.describe = 'do not know debtor'
    
 
        
class S1_N103(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n103_paymentChannel_s', msg_path)
        self.describe = 'notify methods of paying'
    

        
class S1_N104(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n104_paymentChannel_s', msg_path)
        self.describe = 'notify methods of paying'
    
        
class S1_N105(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n105_noResult_s',msg_path)
        self.describe = 'no result'
        
    

class S1_N106(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n106_paymentChannel_s',msg_path)
        self.describe = 'notify methods of paying'
        
        
        
class S1_N108(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n108_noResult_s',msg_path)
        self.describe = 'no result'
        
class S1_N109(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n109_scheduleCall_s',msg_path)
        self.describe = 'no result'
        
class S1_N110(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n110_recordWechat_s',msg_path)
        self.describe = 'no result'
        
    


In [26]:
class TreeBase:
    def __init__(self, start_node='s0', ):
        self.current_node_name = start_node
        self.fc_path = []
        self.all_path = []
        
        
    
class TreeStage1(TreeBase):
    def __init__(self, start_node='s0',graph_path='',msg_path=''):
        super().__init__(start_node=start_node)
        self._build_node(msg_path)
        self._build_graph(graph_path)
        
    def _build_node(self,msg_path):
        self.messages = pd.read_csv(msg_path,encoding='utf8')
        self.nodes = {
        's0':S1_N0(msg_path),
        'cf_s1_n1_identity_q':S1_N1(msg_path),
        'cf_s1_n15_verifyWill_q':S1_N15(msg_path),
        'cf_s1_n101_ifAcquainted_s':S1_N101(msg_path),
        'cf_s1_n102_ifAcquainted_s':S1_N102(msg_path),
        'cf_s1_n103_paymentChannel_s':S1_N103(msg_path),
        'cf_s1_n104_paymentChannel_s':S1_N104(msg_path),
        'cf_s1_n105_noResult_s':S1_N105(msg_path),
        'cf_s1_n106_paymentChannel_s':S1_N106(msg_path),
        'cf_s1_n108_noResult_s':S1_N108(msg_path),
        'cf_s1_n109_scheduleCall_s':S1_N109(msg_path),
        'cf_s1_n110_recordWechat_s':S1_N110(msg_path),
        'cf_s1_n2_confirmLoan_q': S1_N2(msg_path),
        'cf_s1_n25_cutDebt_q':S1_N25(msg_path),
        'cf_s1_n32_splitDebt_q':S1_N32(msg_path),
        'cf_s1_n5_ifAcquainted_q':S1_N5(msg_path),} 
        
    def _build_graph(self,graph_path):
        self.df_mapping = pd.read_csv(graph_path)
        gp = self.df_mapping.groupby('node_name')
        self.mapping = {}
        for each in gp:
            df_tmp = each[1]
            df_tmp = df_tmp.set_index('label')
            self.mapping.update({each[0]:df_tmp.T.to_dict()})
        
        

        
    def _updates(self, _label):
        """
        update fc_path, all_path, current_node_name
        return current node, response
        """
        cur_node = self.nodes[self.current_node_name]
        
        # get current response
        response = cur_node.get_response(_label)
        next_node_name = self.mapping.get(self.current_node_name)
        
        # get next node_name
        if self.mapping.get(self.current_node_name) is not None:
            next_node_name = self.mapping.get(self.current_node_name)[_label]['connection']
        else:
            next_node_name = None
        return response, next_node_name
        
        
        
    def process(self, sentence, model_dict):
        current_node = self.nodes[self.current_node_name] 
        if current_node.model_name == 'StopClassifier':
            return 'end'
        _label,_ptp = current_node.process(sentence, model_dict)

        response,next_node_name = self._updates(_label)
        
        if next_node_name is None:
            return 'end'
        else:
            self.current_node_name = next_node_name
        return response

In [27]:
t1 = TreeStage1(graph_path='../../data/TreeModel/treeConnection.csv',msg_path='../../data/TreeModel/node_message.csv')

s0 is initialized
cf_s1_n1_identity_q is initialized
cf_s1_n15_verifyWill_q is initialized
cf_s1_n101_ifAcquainted_s is initialized
cf_s1_n102_ifAcquainted_s is initialized
cf_s1_n103_paymentChannel_s is initialized
cf_s1_n104_paymentChannel_s is initialized
cf_s1_n105_noResult_s is initialized
cf_s1_n106_paymentChannel_s is initialized
cf_s1_n108_noResult_s is initialized
cf_s1_n109_scheduleCall_s is initialized
cf_s1_n110_recordWechat_s is initialized
cf_s1_n2_confirmLoan_q is initialized
cf_s1_n25_cutDebt_q is initialized
cf_s1_n32_splitDebt_q is initialized
cf_s1_n5_ifAcquainted_q is initialized


In [32]:
print(t1.current_node_name)
print(t1.process('不行',model_dict))
print(t1.current_node_name)

cf_s1_n102_ifAcquainted_s
StopClassifier
end
cf_s1_n102_ifAcquainted_s


In [8]:
t1.messages

node_name  label  sentiment  \
0                        s0      0          1   
1       cf_s1_n1_identity_q      1          1   
2       cf_s1_n1_identity_q      0          1   
3       cf_s1_n1_identity_q    101          1   
4       cf_s1_n1_identity_q    103          1   
5       cf_s1_n1_identity_q    104          1   
6       cf_s1_n1_identity_q    107          1   
7       cf_s1_n1_identity_q    109          1   
8       cf_s1_n1_identity_q    110          1   
9    cf_s1_n2_confirmLoan_q      0          1   
10   cf_s1_n2_confirmLoan_q      1          1   
11   cf_s1_n2_confirmLoan_q    100          1   
12   cf_s1_n2_confirmLoan_q    102          1   
13   cf_s1_n2_confirmLoan_q    103          1   
14   cf_s1_n2_confirmLoan_q    104          1   
15   cf_s1_n2_confirmLoan_q    106          1   
16   cf_s1_n2_confirmLoan_q    107          1   
17   cf_s1_n2_confirmLoan_q    108          1   
18   cf_s1_n2_confirmLoan_q    109          1   
19   cf_s1_n2_confirmLoan_q   1001          1   
20  cf_s1_n5_ifAcquainted_q      0          1   
21  cf_s1_n5_ifAcquainted_q      1          1   
22  cf_s1_n5_ifAcquainted_q    101          1   
23  cf_s1_n5_ifAcquainted_q    103          1   
24  cf_s1_n5_ifAcquainted_q    104          1   
25  cf_s1_n5_ifAcquainted_q    107          1   
26  cf_s1_n5_ifAcquainted_q    109          1   
27  cf_s1_n5_ifAcquainted_q    110          1   
28   cf_s1_n15_verifyWill_q      0          1   
29   cf_s1_n15_verifyWill_q      1          1   
..                      ...    ...        ...   
31   cf_s1_n15_verifyWill_q    103          1   
32   cf_s1_n15_verifyWill_q    104          1   
33   cf_s1_n15_verifyWill_q    105          1   
34   cf_s1_n15_verifyWill_q    106          1   
35   cf_s1_n15_verifyWill_q    107          1   
36   cf_s1_n15_verifyWill_q    108          1   
37   cf_s1_n15_verifyWill_q    109          1   
38   cf_s1_n15_verifyWill_q   1001          1   
39      cf_s1_n25_cutDebt_q      0          1   
40      cf_s1_n25_cutDebt_q      1          1   
41      cf_s1_n25_cutDebt_q    100          1   
42      cf_s1_n25_cutDebt_q    102          1   
43      cf_s1_n25_cutDebt_q    103          1   
44      cf_s1_n25_cutDebt_q    104          1   
45      cf_s1_n25_cutDebt_q    106          1   
46      cf_s1_n25_cutDebt_q    107          1   
47      cf_s1_n25_cutDebt_q    108          1   
48      cf_s1_n25_cutDebt_q    109          1   
49      cf_s1_n25_cutDebt_q   1001          1   
50    cf_s1_n32_splitDebt_q      0          1   
51    cf_s1_n32_splitDebt_q      1          1   
52    cf_s1_n32_splitDebt_q    100          1   
53    cf_s1_n32_splitDebt_q    102          1   
54    cf_s1_n32_splitDebt_q    103          1   
55    cf_s1_n32_splitDebt_q    104          1   
56    cf_s1_n32_splitDebt_q    106          1   
57    cf_s1_n32_splitDebt_q    107          1   
58    cf_s1_n32_splitDebt_q    108          1   
59    cf_s1_n32_splitDebt_q    109          1   
60    cf_s1_n32_splitDebt_q   1001          1   

                                              message  
0                            你好，这里是江苏逸能催收公司，请问是罗巍先生吗？  
1                                 不好意思，打扰了，请问您认识罗先生吗！  
2   您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500...  
3   是这样的，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息15...  
4                           我们这里是江苏逸能催收公司的，请问您是罗巍先生吗？  
5                                那好吧，那我过一会儿再给您打过去。再见。  
6   我这边是江苏逸能催收公司的催收员。您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5...  
7                                请你不要说这些有的没的，您是罗巍先生吗？  
8         你说这些乱七八糟的都没有用的，这个该还的你肯定是跑不掉的。你现在告诉我你是罗巍先生吗？  
9   您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳...  
10  我们这边所有记录都显示，你找我们借款现在还未还，这个欠了钱不还是不可能的。你这到底是什么原因...  
11  嗯，这样的话我这边可以帮你申请一下，看能不能把利息和滞纳金给你去掉，只需要还5万，但是你必须...  
12  这些app上每一笔都是可以看到的，利息1500，滞纳金500，本金5万。你这些钱一直不处理会...  
13  是这样的，您之前借的5万块已经逾期了47天了，现在有1500利息和500块滞纳金，总共需要还...  
14                               那好吧，那我过一会儿再给您打过去。再见。  
15                        你这模模糊糊的是什么意思？到底是为什么说清楚不就好了。  
16                跟你说了我是江苏逸

In [37]:
t1.nodes['cf_s1_n2_confirmLoan_q'].get_response(2)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
print(t1.process('是你大爷',model_dict))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
b = t1.nodes['s0'].messages
b

node_name  label sentiment                   message
0        s0      0         1  你好，这里是江苏逸能催收公司，请问是罗巍先生吗？

In [ ]:
def get_response(self, label, sentiment=1):
        """
        return response by label
        """
        df = self.messages[(self.messages.label == label) & (self.messages.sentiment == sentiment)]
        response = df.message.values[0]
        self.entry_counter += 1
        return response

In [30]:
model_dict['IfKnowDebtor'].classify('可以')

{'label': 104, 'pred_prob': array([[0.39139882, 0.10326868, 0.5053325 ],
        [0.43655833, 0.18918957, 0.3742521 ],
        [0.5465552 , 0.21296955, 0.24047525]]), 'av_pred': array([0.45817078, 0.16847594, 0.37335328])}